In [ ]:
# client.py
import webbrowser
import http.server
import socketserver
import threading
import json

PORT = 8001
json_results = None

HTML_CONTENT = r"""
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>Beauty Product Recommendation</title>

<style>
    body {
        background:#fafafa;
        font-family:Arial;
        margin:0;
        padding:0;
    }

    header {
        background:#4C7AF2;
        padding:20px;
        color:white;
        font-size:24px;
        text-align:center;
        font-weight:bold;
    }

    .container {
        width:90%;
        max-width:900px;
        margin:auto;
        padding:20px;
    }

    .section-title {
        font-size:20px;
        font-weight:bold;
        margin-top:20px;
        margin-bottom:12px;
    }

    /* 三列布局：字段名 | 输入框 | Hard */
    .grid-row {
        display:grid;
        grid-template-columns: 140px 240px 100px;
        gap: 10px;
        align-items:center;
        margin-bottom:10px;
    }

    label.field-label {
        font-size: 14px;
        font-weight: bold;
        white-space: nowrap;
    }

    input, textarea, select {
        width: 100%;
        padding: 8px;
        border-radius: 6px;
        border: 1px solid #ccc;
        font-size: 14px;
    }

    textarea { height: 70px; }

    /* Hard 统一样式 */
    .hard-flag-box {
        display: flex;
        align-items: center;
        font-size: 14px;
        white-space: nowrap;
    }
    .hard-flag-box input {
        margin-right: 6px;
    }

    /* 结果卡片 */
    .results-grid {
        margin-top: 30px;
        display: grid;
        grid-template-columns: repeat(auto-fill, minmax(260px, 1fr));
        gap: 18px;
    }

    .card {
        background: white;
        border-radius: 10px;
        padding: 14px;
        box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        transition: transform .2s;
    }

    .card:hover {
        transform: translateY(-4px);
    }

    .card img {
        width: 100%;
        height: 200px;
        object-fit: contain;
        border-radius: 8px;
        background: #fff;
    }

    .card-title {
        font-weight: bold;
        margin-top: 8px;
    }

    .score {
        color: #4C7AF2;
        font-size: 14px;
        margin-top: 4px;
    }

    .summary {
        font-size: 13px;
        color: #444;
        margin-top: 8px;
        max-height: 120px;
        overflow: auto;
    }

</style>
</head>

<body>
<header>Beauty Product Recommendation System</header>

<div class="container">

    <div class="section-title">Query</div>
    <textarea id="query" placeholder="Describe what you're looking for"></textarea>

    <!-- ================= CONSTRAINTS ================= -->
    <div class="section-title">Constraints & Settings</div>

    <div class="grid-row">
        <label class="field-label">Brand</label>
        <input id="brand" type="text">
        <label class="hard-flag-box"><input id="hard_brand" type="checkbox"> Hard</label>
    </div>

    <div class="grid-row">
        <label class="field-label">Price Min</label>
        <input id="price_min" type="number">
        <label class="hard-flag-box"><input id="hard_price_min" type="checkbox"> Hard</label>
    </div>

    <div class="grid-row">
        <label class="field-label">Price Max</label>
        <input id="price_max" type="number">
        <label class="hard-flag-box"><input id="hard_price_max" type="checkbox"> Hard</label>
    </div>

    <div class="grid-row">
        <label class="field-label">Rating Min</label>
        <input id="rating_min" type="number">
        <label class="hard-flag-box"><input id="hard_rating_min" type="checkbox"> Hard</label>
    </div>

    <div class="grid-row">
        <label class="field-label">Top-N</label>
        <input id="topn" type="number">
        <div></div>
    </div>

    <div class="grid-row">
        <label class="field-label">PPR@K</label>
        <input id="ppr_k" type="number">
        <div></div>
    </div>

    <!-- ================= ATTRIBUTES ================= -->
    <div class="section-title">Attributes</div>

    <script>
        const vocab = {
            item_form: ["cream","liquid","gel","pair","powder","spray","oil","bar","lotion","pencil","stick","wand","balm","wrap","scrunchie","individual","elastic","sheet","clay","serum","foam","wax","butter","clip","wipes","spiral","ribbon","mask","aerosol","strip"],
            material: ["human hair","synthetic","plastic","acrylic","human","metal","cotton","rubber","faux mink","silicone","silk","ceramic","nylon","polyester","mink fur","stainless steel","wood","acrylonitrile butadiene styrene (abs)"],
            hair_type: ["straight","wavy","curly","kinky","coily","all","dry","thick","fine","normal","frizzy","color","damaged"],
            age_range: ["adult","kid","child","baby","all ages"],
            material_feature: ["natural","cruelty free","organic","latex free","non-toxic","reusable","vegan","disposable","biodegradable warning","gluten free","certified organic"],
            color: ["black","pink","white","blue","brown","red","natural","clear","gold","silver","green","purple","multicolor","beige"],
            skin_type: ["all","sensitive","dry","acne prone","oily","normal","combination"],
            style: ["modern","french","straight","compact","curly","african","classic","art deco","wavy","earloop"]
        };
    </script>

    <div id="attr-container"></div>

    <script>
        function buildAttributes() {
            const container = document.getElementById("attr-container");
            const keys = Object.keys(vocab);

            keys.forEach(key => {
                const row = document.createElement("div");
                row.className = "grid-row";

                const label = document.createElement("label");
                label.className = "field-label";
                label.textContent = key;

                const sel = document.createElement("select");
                sel.id = key;

                const opt0 = document.createElement("option");
                opt0.value = "";
                opt0.textContent = "(none)";
                sel.appendChild(opt0);

                vocab[key].forEach(v => {
                    const o = document.createElement("option");
                    o.value = v;
                    o.textContent = v;
                    sel.appendChild(o);
                });

                const hard = document.createElement("label");
                hard.className = "hard-flag-box";
                hard.innerHTML = `<input type="checkbox" id="hard_${key}"> Hard`;

                row.appendChild(label);
                row.appendChild(sel);
                row.appendChild(hard);

                container.appendChild(row);
            });
        }
        buildAttributes();
    </script>

    <button onclick="sendRequest()">Recommend</button>

    <h3 style="margin-top:28px;">Results</h3>
    <div id="results" class="results-grid"></div>

</div>

<!-- ================= JS ================= -->
<script>
async function sendRequest() {
    const constraints = {
        brand: document.getElementById("brand").value || null,
        price_min: parseFloat(document.getElementById("price_min").value) || null,
        price_max: parseFloat(document.getElementById("price_max").value) || null,
        rating_min: parseFloat(document.getElementById("rating_min").value) || null,
    };

    const hard_flags = {
        brand: document.getElementById("hard_brand").checked,
        price_min: document.getElementById("hard_price_min").checked,
        price_max: document.getElementById("hard_price_max").checked,
        rating_min: document.getElementById("hard_rating_min").checked
    };

    const attrKeys = Object.keys(vocab);
    attrKeys.forEach(key => {
        const val = document.getElementById(key).value;
        if (val) constraints[key] = val;
        hard_flags[key] = document.getElementById("hard_" + key).checked;
    });

    const payload = {
        query: document.getElementById("query").value,
        constraints: constraints,
        hard_flags: hard_flags,
        topn: parseInt(document.getElementById("topn").value),
        ppr_k: parseInt(document.getElementById("ppr_k").value)
    };

    const resultsDiv = document.getElementById("results");
    resultsDiv.innerHTML = "Loading...";

    const res = await fetch("http://127.0.0.1:8000/recommend", {
        method:"POST",
        headers:{"Content-Type":"application/json"},
        body:JSON.stringify(payload)
    });

    const json = await res.json();

    // send results back to Python to store
    await fetch("http://127.0.0.1:8001/save", {
        method: "POST",
        headers: {"Content-Type": "application/json"},
        body: JSON.stringify(json)
    });

    if (!json.results || json.results.length === 0) {
        resultsDiv.innerHTML = "<p>No results.</p>";
        return;
    }

    resultsDiv.innerHTML = "";
    json.results.forEach(item => {
        const card = document.createElement("div");
        card.className = "card";
        card.innerHTML = `
            <img src="${item.image_url || ""}">
            <div class="card-title">${item.title}</div>
            <div class="score">Score: ${item.scores.fused.toFixed(3)}</div>
            <div class="summary">${item.summary || "(no summary)"}</div>
        `;
        resultsDiv.appendChild(card);
    });
}
</script>

</body>
</html>
"""

class Handler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type","text/html")
        self.end_headers()
        self.wfile.write(HTML_CONTENT.encode("utf-8"))

    def do_POST(self):
        global json_results
        if self.path == "/save":
            length = int(self.headers.get("Content-Length",0))
            post_data = self.rfile.read(length)
            json_results = json.loads(post_data.decode("utf-8"))
            print(json.dumps(json_results, indent=2, ensure_ascii=False))
            self.send_response(200)
            self.end_headers()

def start_server():
    with socketserver.TCPServer(("", PORT), Handler) as httpd:
        print(f"Serving client at http://127.0.0.1:{PORT}")
        httpd.serve_forever()

if __name__ == "__main__":
    threading.Thread(target=start_server, daemon=True).start()
    webbrowser.open(f"http://127.0.0.1:{PORT}")
    input("Press Enter to exit...\n")
    print("Saved json_results:")
    print(json_results)
